In [2]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
import torch
from transformers import BertTokenizer, BertModel
import matplotlib.pyplot as plt
import pickle
import time
import numpy as np
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
from scipy import stats
import random
from math import floor

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jafar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import psycopg2
con = psycopg2.connect(database="postgres", user="postgres", password="Jafarsql", host="localhost", port="5432")
print("Database opened successfully")

cur = con.cursor()
q='''SELECT screen_name, tweet FROM manager_world_tweets_ph2 
    Where 
    tweet_created_at like '%2019%';'''
cur.execute(q)
rows_mng = cur.fetchall()
# there is no repetative tweet
con.close()

print(len(rows_mng))

Database opened successfully
5983238


In [4]:
import psycopg2
con = psycopg2.connect(database="postgres", user="postgres", password="Jafarsql", host="localhost", port="5432")
print("Database opened successfully")

cur = con.cursor()
q='''SELECT screen_name, tweet FROM ent_world_tweets_ph2 
    Where 
    tweet_created_at like '%2019%';'''
cur.execute(q)
rows_ent = cur.fetchall()
# there is no repetative tweet
con.close()

print(len(rows_ent))

Database opened successfully
5237437


In [5]:
from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
Stem=stemmer.stem

In [6]:
def cleaning (text):
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    #removing emoji
    text = emoji_pattern.sub(r' ', text)   

    #removeing http and https (URL)
    text = re.sub(r'(http://|https://)\S+', '', text)
    
    #removing www (URL)
    text=re.sub(r'www\.\S+', '', text)
    
    #removing targets
    text=re.sub('( |^)@\S+', '', text) 
    
    #removing tabs and lines
    text=re.sub('\t|\n', ' ', text)

    #removing some special charachter  
    text= re.sub("[\"\“\”\+\-\|\*\?\(\)\/\\\^\[\]\.{}_`;•«,@:~!=%&]+", ' ',text) # except ' ’
    
    #removing hashtag sign
    text=re.sub('#', '', text) 
    
    #removing numbers
    text=re.sub("(\d+)",' ',text)
#    print(text)
    #removing numbers (not attached)
#    text=re.sub(r"(^|\s)-?(\d+)\.?(\d*)(\s)", '', text)
#    text=re.sub("(^||\d+)\.(\d+) ",'',text)
#    text=re.sub("[0-9+]\.?(\d+)(\.)",' ',text)
#    text=re.sub("[0-9+]\.?(\d+)($)",' ',text)
#    print(text)
    
#    return text

#def lower_case (text):
    text = text.lower() 
#    print(text)
#    return text_lower

#def tokenization (text):
    text= nltk.word_tokenize(text)
#    print (text)
#    return text_tokens

#def removing_stopwords (text):
    #text_without_sw = [word.lower() for word in text if word.lower() not in stopwords.words()]
#    text = [word for word in text if word not in stopwords.words()]
    text = [word for word in text if word not in cachedStopWords]
#    print(text)
#    return text_without_sw

#def stemming (text):
    #stemmer = SnowballStemmer("english")
    #text_stems=stemmer.stem(tokens_without_sw)
#    text = [stemmer.stem(word) for word in text]
    text = [Stem(word) for word in text]

#    print(text)
    
    text=' '.join(text)
#    print(text)
#    print(text,'\n')   
    
    return text

In [7]:
names_ent=[]
for row in rows_ent:
    names_ent.append(row[0])

In [8]:
names_mng=[]
for row in rows_mng:
    names_mng.append(row[0])

In [9]:
mng_cleaned=[]
for row in rows_mng:
    ans=cleaning(row[1])
    mng_cleaned.append(ans)

In [10]:
ent_cleaned=[]
for row in rows_ent:
    ans=cleaning(row[1])
    ent_cleaned.append(ans)

In [11]:
with open("tweet_mng5M_cleaned.txt", "wb") as fp:   #Pickling
  pickle.dump(mng_cleaned, fp)

with open("tweet_ent5M_cleaned.txt", "wb") as fp:   #Pickling
  pickle.dump(ent_cleaned, fp)

with open("names_mng5M.txt", "wb") as fp:   #Pickling
  pickle.dump(names_mng, fp)

with open("names_ent5M.txt", "wb") as fp:   #Pickling
  pickle.dump(names_ent, fp)

In [106]:
'''
mng_cleaned=[]
for tweet in rows_mng:
    ans=cleaning(tweet[0])
    if ans!=[]:
        mng_cleaned.append(ans)
'''        

'\nmng_cleaned=[]\nfor tweet in rows_mng:\n    ans=cleaning(tweet[0])\n    if ans!=[]:\n        mng_cleaned.append(ans)\n'

In [107]:
'''
ent_cleaned=[]
for tweet in rows_ent:
    ans=cleaning(tweet[0])
    if ans!=[]:
        ent_cleaned.append(ans)
'''        

'\nent_cleaned=[]\nfor tweet in rows_ent:\n    ans=cleaning(tweet[0])\n    if ans!=[]:\n        ent_cleaned.append(ans)\n'

In [108]:
#rows_ent[0][0]

In [109]:
ent_words=[]
for tweet in ent_cleaned:
    ent_words += tweet.split()

In [110]:
mng_words=[]
for tweet in mng_cleaned:
    mng_words += tweet.split()

For the similarity between two datasets we have

In [133]:
chi2=[]
p_value=[]
ent_size=2600000
mng_size=2600000
most_frequent=10

for kk in range(10):
    
#    ent_size=floor(len(ent_words)/2)
    ent_words_random=random.sample(ent_words,ent_size)
    
#    mng_size=floor(len(mng_words)/2)
    mng_words_random=random.sample(mng_words,mng_size)
    
    all_words=mng_words_random+ent_words_random


    d_mng = dict()
    for c in mng_words_random:
        if c not in d_mng:
            d_mng[c] = 1
        else:
            d_mng[c] = d_mng[c] + 1

    d_mng_new=dict(sorted(d_mng.items(), key=lambda x: x[1], reverse=True))
    key_list_mng=list(d_mng_new.keys())
    values_list_mng=list(d_mng_new.values())
    frequent_words_mng=set(list(d_mng_new.keys())[0:most_frequent])
#    print(list(d_mng_new.keys())[0:most_frequent])
#    print(list(d_mng_new.values())[0:most_frequent])        

    d_ent = dict()
    for c in ent_words_random:
        if c not in d_ent:
            d_ent[c] = 1
        else:
            d_ent[c] = d_ent[c] + 1

    d_ent_new=dict(sorted(d_ent.items(), key=lambda x: x[1], reverse=True))
    key_list_ent=list(d_ent_new.keys())
    values_list_ent=list(d_ent_new.values())
    frequent_words_ent=set(list(d_ent_new.keys())[0:most_frequent])
#    print(list(d_ent_new.keys())[0:most_frequent])
#    print(list(d_ent_new.values())[0:most_frequent])       

    d_all = dict()
    for c in all_words:
        if c not in d_all:
            d_all[c] = 1
        else:
            d_all[c] = d_all[c] + 1

    d_all_new=dict(sorted(d_all.items(), key=lambda x: x[1], reverse=True))
    key_list_all=list(d_all_new.keys())
    values_list_all=list(d_all_new.values())
#    print(list(d_all_new.keys())[0:most_frequent])
#    print(list(d_all_new.values())[0:most_frequent])        

#    print(len(key_list_all))
#    print(len(key_list_mng))
#    print(len(key_list_ent))

    frequent=key_list_all[0:most_frequent] # 
    #frequent

    text1_freq_values=[]
    text2_freq_values=[]

    for ii in frequent:
    #    print(ii)
    #    print(d_ent_new[ii])
    #    print(d_mng_new[ii])
        try: 
            text1_freq_values.append(d_ent_new[ii])
        except:
            text1_freq_values.append(0)
        try:    
            text2_freq_values.append(d_mng_new[ii])
        except:
            text2_freq_values.append(0)
            
            
#        except:
#            print(ii)
#            print(d_mng_new[ii])
#            print(d_ent_new[ii])

    #print(text1_freq_values )  
    #print(text2_freq_values)    

    chi2_stat, p_val, dof, ex = stats.chi2_contingency([text1_freq_values,text2_freq_values])
#    print(chi2_stat)
#    print(p_val)
    chi2.append(chi2_stat)
    p_value.append(p_val)

For 10 words we have

In [134]:
print("intersection :", frequent_words_mng & frequent_words_ent)
print()
print("mng - ent :",frequent_words_mng - frequent_words_ent)
print()
print("ent- mng :",frequent_words_ent - frequent_words_mng)

intersection : {'go', 'rt', 'get', 'amp', "'s", "n't", 'like', 'one', '’'}

mng - ent : {'time'}

ent- mng : {'peopl'}


it is possible that, for example, "time" is 11th word for ent and "peopl" (people) is 12th for mng.

For 500 words we have (since the number of intersetion is high, we just show its length)

In [131]:
print("length of intersection :", len (frequent_words_mng & frequent_words_ent))
print()
print("mng - ent :",frequent_words_mng - frequent_words_ent)
print()
print("ent- mng :",frequent_words_ent - frequent_words_mng)

length of intersection : 424

mng - ent : {'beat', 'age', 'eye', 'footbal', 'dog', 'record', 'honest', 'releas', "'d", 'absolut', 'yesterday', 'near', 'uk', 'view', 'episod', 'im', 'pretti', 'pictur', '£', 'updat', 'match', 'guess', '\u2066', 'friday', 'fantast', 'huge', 'light', 'probabl', 'cover', 'luck', 'appreci', 'least', 'class', 'abl', 'vs', 'pick', 'liter', 'leagu', 'a…', 'quit', 'congrat', 'christma', 'late', 'short', 'took', 'finish', 'hell', 'three', 'local', 'coach', 'club', 'road', 'brexit', 'saw', 'goe', 'favorit', 't…', 'heard', 'mate', 'movi', 'star', 'ticket', 'stream', 'special', 'stuff', 'perform', 'drink', 'sorri', 'cool', 'remind', 'half', 'sunday', 'super', 'almost', 'bit', 'photo'}

ent- mng : {'network', 'focus', 'startup', 'technolog', 'ass', 'nigga', 'space', 'act', 'googl', 'trust', 'water', 'dear', 'daili', 'energi', 'digit', 'tip', 'travel', 'spend', 'truth', 'sell', 'entrepreneur', 'ai', 'provid', 'key', 'phone', 'brand', 'sale', 'connect', 'peac', 'podcas

For 1000 words we have

In [129]:
print("length of intersection :", len (frequent_words_mng & frequent_words_ent))
print()
print("mng - ent :",frequent_words_mng - frequent_words_ent)
print()
print("ent- mng :",frequent_words_ent - frequent_words_mng)

length of intersection : 876

mng - ent : {'impress', 'blue', 'appear', '£', 'h…', 'cold', 'normal', 'ride', 'johnson', 'tori', 'pre', 'afternoon', 'bed', 'whether', 'sweet', 'cup', 'juli', 'suggest', 'arriv', 'lt', 'liverpool', 'v', 'danc', 'involv', 'sick', 'favourit', 'scene', 'eu', 'racist', 'fm', 'extra', 'usual', 'ice', 'recommend', 'staff', 'champion', 'station', 'obvious', 'legend', 'main', 'chat', 'charact', 'i…', 'labour', 'battl', 'bori', 'weather', 'suppos', 'field', 'bought', 'haha', 'massiv', 'memori', 'weird', 'sent', 's…', 'brilliant', 'spent', 'notic', 'sever', 'roll', 'kick', 'pack', 'edit', 'tour', 'anyway', 'giveaway', 'rain', 'seat', 'west', 'cheer', 'bar', 'titl', 'loss', 'omg', 'cute', 'four', 'defend', 'rd', 'awar', 'cat', 'basic', 'shirt', 'coffe', 'mate', 'lucki', 'plus', 'channel', 'assist', 'coupl', 'version', 'trip', 'straight', 'march', 'except', 'five', 'hospit', 'switch', 'sort', 'met', 'town', 'draw', 'fantast', 'contact', 'race', 'twitch', 'hang', 'don

For 10000 words we have

In [122]:
print("length of intersection :", len (frequent_words_mng & frequent_words_ent))
print("length of mng - ent :",len(frequent_words_mng - frequent_words_ent)
print("length of ent- mng :",len(frequent_words_ent - frequent_words_mng))

length of intersection : 8194
length of mng - ent : 1806
length of ent- mng : 1806


For 100000 words we have

In [120]:
print("length of intersection :", len (frequent_words_mng & frequent_words_ent))
print("length of mng - ent :",len(frequent_words_mng - frequent_words_ent))
print("length of ent- mng :",len(frequent_words_ent - frequent_words_mng))

length of intersection : 40149
length of mng - ent : 59851
length of ent- mng : 59851


for 100000 most frequent words we have

In [26]:

CHI2_mean=np.mean(chi2)
CHI2_std=np.std(chi2)
P_VALUE_mean=np.mean(p_value)
P_VALUE_std=np.std(p_value)

print('chi2:',chi2)
print('CHI2_mean:',CHI2_mean)
print('CHI2_std:',CHI2_std)
print('p_value:',p_value)
print('P_VALUE_mean:',P_VALUE_mean)
print('P_VALUE_std:',P_VALUE_std)

intersection: {'go', 'rt', 'get', 'amp', "'s", "n't", 'like', 'one', '’'}
ent_words - mng_words: {'peopl'}
mng_words - ent_words: {'time'}

chi2: [1028.7213068384015]
CHI2_mean: 1028.7213068384015
CHI2_std: 0.0
p_value: [1.1034656798308345e-215]
P_VALUE_mean: 1.1034656798308345e-215
P_VALUE_std: 0.0


for 10000 most frequent words we have

In [26]:
CHI2_mean=np.mean(chi2)
CHI2_std=np.std(chi2)
P_VALUE_mean=np.mean(p_value)
P_VALUE_std=np.std(p_value)

print('chi2:',chi2)
print('CHI2_mean:',CHI2_mean)
print('CHI2_std:',CHI2_std)
print('p_value:',p_value)
print('P_VALUE_mean:',P_VALUE_mean)
print('P_VALUE_std:',P_VALUE_std)

chi2: [144205.29735274083, 143188.48437141583, 143026.9569510265, 142381.70934693966, 143496.301480799, 142888.19713576583, 143488.67315995716, 143717.22011785215, 143189.74457660157, 143612.69451357206]
CHI2_mean: 143319.52790066708
CHI2_std: 476.83498706944044
p_value: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
P_VALUE_mean: 0.0
P_VALUE_std: 0.0


for 1000 most frequent words we have

In [12]:
CHI2_mean=np.mean(chi2)
CHI2_std=np.std(chi2)
P_VALUE_mean=np.mean(p_value)
P_VALUE_std=np.std(p_value)

print('chi2:',chi2)
print('CHI2_mean:',CHI2_mean)
print('CHI2_std:',CHI2_std)
print('p_value:',p_value)
print('P_VALUE_mean:',P_VALUE_mean)
print('P_VALUE_std:',P_VALUE_std)

chi2: [59496.93504468392, 60015.64350626074, 59976.72548266546, 59131.16002254333, 59797.948359375616, 59825.33408178453, 59586.13962031849, 59705.060259595106, 59730.0892863322, 59949.97237651255]
CHI2_mean: 59721.500804007206
CHI2_std: 252.60307316474265
p_value: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
P_VALUE_mean: 0.0
P_VALUE_std: 0.0


For example, for the last iteration, we have the following most 20 frequent words:

In [13]:
print('\n 20 most frequnt words in ent random corpus:')
print(key_list_ent[0:20])

print('\n 20 most frequnt words in mng random corpus:')
print(key_list_mng[0:20])


 20 most frequnt words in ent random corpus:
['rt', '’', "'s", "n't", 'get', 'like', 'amp', 'one', 'peopl', 'go', 'time', 'make', 'know', 'thank', 'need', 'day', 'year', 'love', 'new', 'see']

 20 most frequnt words in mng random corpus:
['rt', '’', "'s", 'get', 'like', "n't", 'one', 'go', 'time', 'amp', 'day', 'thank', 'year', 'peopl', 'good', 'see', 'love', 'look', 'new', 'make']


For 500 frequent words we have:

In [31]:
CHI2_mean=np.mean(chi2)
CHI2_std=np.std(chi2)
P_VALUE_mean=np.mean(p_value)
P_VALUE_std=np.std(p_value)

print('chi2:',chi2)
print('CHI2_mean:',CHI2_mean)
print('CHI2_std:',CHI2_std)
print('p_value:',p_value)
print('P_VALUE_mean:',P_VALUE_mean)
print('P_VALUE_std:',P_VALUE_std)

chi2: [40424.90370163364, 41189.86804521994, 41244.87607402078, 40336.55179638849, 40659.135096578415, 41057.345171925364, 42081.75096789941, 40782.03615662649, 41126.05020491114, 41119.67360871063]
CHI2_mean: 41002.21908239143
CHI2_std: 472.6780474809556
p_value: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
P_VALUE_mean: 0.0
P_VALUE_std: 0.0


For example, for the last iteration, we have the following most 20 frequent words:

In [32]:
print('\n 20 most frequnt words in ent random corpus:')
print(key_list_ent[0:20])

print('\n 20 most frequnt words in mng random corpus:')
print(key_list_mng[0:20])


 20 most frequnt words in ent random corpus:
['rt', '’', "'s", "n't", 'get', 'like', 'amp', 'one', 'peopl', 'go', 'time', 'make', 'know', 'thank', 'need', 'day', 'see', 'year', 'new', 'love']

 20 most frequnt words in mng random corpus:
['rt', '’', "'s", 'get', 'like', "n't", 'one', 'go', 'time', 'amp', 'day', 'peopl', 'thank', 'year', 'good', 'love', 'see', 'look', 'make', 'know']


For 10 frequent words we have:

In [83]:
CHI2_mean=np.mean(chi2)
CHI2_std=np.std(chi2)
P_VALUE_mean=np.mean(p_value)
P_VALUE_std=np.std(p_value)

print('chi2:',chi2)
print('CHI2_mean:',CHI2_mean)
print('CHI2_std:',CHI2_std)
print('p_value:',p_value)
print('P_VALUE_mean:',P_VALUE_mean)
print('P_VALUE_std:',P_VALUE_std)

chi2: [845.7593821951187, 781.0033209526467, 702.422465411873, 778.3694323543484, 818.3645986097752, 758.3394052524558, 694.3793855552625, 738.3464935632435, 734.7549451568664, 804.2575955394715]
CHI2_mean: 765.5997024591062
CHI2_std: 46.81348509496665
p_value: [2.988127514447561e-176, 2.607470991490118e-162, 2.0848843459183727e-145, 9.616996779647768e-162, 2.3668546601682847e-170, 1.9632727264313904e-157, 1.1172732610576256e-143, 3.925394462975115e-153, 2.3248042658715486e-152, 2.576932030903781e-167]
P_VALUE_mean: 1.1381221072341727e-144
P_VALUE_std: 3.345447684616233e-144


For example, for the last iteration, for the most 10 frequent words, we have:

In [84]:

print('\n 10 most frequent words in ent random corpus:')
print(key_list_ent[0:10])

print('\n 10 most frequent words in mng random corpus:')
print(key_list_mng[0:10])



 10 most frequent words in ent random corpus:
['rt', '’', "'s", "n't", 'get', 'like', 'amp', 'one', 'peopl', 'go']

 10 most frequent words in mng random corpus:
['rt', '’', "'s", 'get', 'like', "n't", 'one', 'go', 'time', 'amp']


**Null hypothesis**: The null hypothesis states that the proportion of words in ent is identical to the proportion of words in mng, or corpuses are similar

**Interpreting results**: Since the P-VALUE, P($\chi^2$> chi2), is less than the significance level (0.05), we reject the null hypothesis.
So, for both 10 and 500 frequent words, the test says that corpuses are not similar.

**For *intra similarity* for mng set (based on random selection of tweets) we have**

In [193]:
chi2=[]
p_value=[]

Mng1_words=mng_words[:floor(len(mng_words)/2)]
Mng2_words=mng_words[floor(len(mng_words)/2):]

most_frequent=10

for kk in range(10):
  
    mng1_words_random=random.sample(Mng1_words,floor(len(Mng1_words)/2))
    mng2_words_random=random.sample(Mng2_words,floor(len(Mng2_words)/2))   
    all_words=mng1_words_random+mng2_words_random
    
#    rang=range(len(mng_words))
#    slice1_ind=random.sample(rang,floor(len(mng_words)/4))
#    temp=list( set(rang) - set(slice1_ind)  ) # list(a.diference(b))
#    slice2_ind=random.sample(temp,floor(len(temp)/3))
    
#    mng1_words_random=[]
#    for ii in slice1_ind:
#        mng1_words_random.append(mng_words[ii])
        
#    mng2_words_random=[]
#    for ii in slice2_ind:
#        mng2_words_random.append(mng_words[ii])
        
#    all_words=mng_words
    
    
    d_mng1 = dict()
    for c in mng1_words_random:
        if c not in d_mng1:
            d_mng1[c] = 1
        else:
            d_mng1[c] = d_mng1[c] + 1

    d_mng1_new=dict(sorted(d_mng1.items(), key=lambda x: x[1], reverse=True))
    key_list_mng1=list(d_mng1_new.keys())
    values_list_mng1=list(d_mng1_new.values())
    frequent_words_mng1=set(list(d_mng1_new.keys())[0:most_frequent])

#    print(list(d_mng1_new.keys())[0:most_frequent])
#    print(list(d_mng1_new.values())[0:most_frequent]) 
#    print()
    
    d_mng2 = dict()
    for c in mng2_words_random:
        if c not in d_mng2:
            d_mng2[c] = 1
        else:
            d_mng2[c] = d_mng2[c] + 1
            
#    print(list(d_mng2_new.keys())[0:most_frequent])
#    print(list(d_mng2_new.values())[0:most_frequent])   
#    print()

    d_mng2_new=dict(sorted(d_mng2.items(), key=lambda x: x[1], reverse=True))
    key_list_mng2=list(d_mng2_new.keys())
    values_list_mng2=list(d_mng2_new.values())
    frequent_words_mng2=set(list(d_mng2_new.keys())[0:most_frequent])

#    print(list(d_all_new.keys())[0:most_frequent])
#    print(list(d_all_new.values())[0:most_frequent]) 
#    print()

    d_all = dict()
    for c in all_words:
        if c not in d_all:
            d_all[c] = 1
        else:
            d_all[c] = d_all[c] + 1

    d_all_new=dict(sorted(d_all.items(), key=lambda x: x[1], reverse=True))
    key_list_all=list(d_all_new.keys())
    values_list_all=list(d_all_new.values())
        

    frequent=key_list_all[0:most_frequent] # 
#    print(frequent)


    text1_freq_values=[]
    text2_freq_values=[]

    for ii in frequent:

        try: 
            text1_freq_values.append(d_mng1_new[ii])
        except:
            text1_freq_values.append(0)
        try:    
            text2_freq_values.append(d_mng2_new[ii])
        except:
            text2_freq_values.append(0)
        
        
#    print(text1_freq_values)  
#    print(text2_freq_values)  

    chi2_stat, p_val, dof, ex = stats.chi2_contingency([text1_freq_values,text2_freq_values])
    chi2.append(chi2_stat)
    p_value.append(p_val)

For 10 most frequent words we have

In [194]:
print("intersection :", frequent_words_mng1 & frequent_words_mng2)
print()
print("mng1 - mng2 :",frequent_words_mng1 - frequent_words_mng2)
print()
print("mng2 - mng1 :",frequent_words_mng2 - frequent_words_mng1)

intersection : {'go', 'time', 'rt', 'get', 'amp', "'s", "n't", 'like', 'one', '’'}

mng1 - mng2 : set()

mng2 - mng1 : set()


For 500 most frequent words we have

In [192]:
print("length of intersection :", len(frequent_words_mng1 & frequent_words_mng2))
print()
print("mng1 - mng2 :",frequent_words_mng1 - frequent_words_mng2)
print()
print("mng2 - mng1 :",frequent_words_mng2 - frequent_words_mng1)

length of intersection : 484

mng1 - mng2 : {'congrat', 'forget', 'american', '\u2066', 'short', 'articl', 'cover', 'hell', 'drink', 'travel', 'price', 'secur', 'near', 'liter', 'room', 'goe'}

mng2 - mng1 : {'saturday', 'sport', 'act', 'light', 'discuss', 'london', 'water', 'road', 'respons', 'inspir', 'bless', 'app', 'incred', 'blog', 'reach', 'brand'}


For 1000 most frequent words we have

In [190]:
print("length of intersection :", len(frequent_words_mng1 & frequent_words_mng2))
print()
print("mng1 - mng2 :",frequent_words_mng1 - frequent_words_mng2)
print()
print("mng2 - mng1 :",frequent_words_mng2 - frequent_words_mng1)

length of intersection : 963

mng1 - mng2 : {'https', 'felt', 'switch', 'david', 'fm', 'click', 'aw', 'middl', 'nigerian', 'gun', 'progress', 'ride', 'hang', 'ps', 'feed', 'bitch', 'stock', 'china', 'trend', 'whether', 'cute', 'tag', 'wast', 'volunt', 'battl', 'quot', 'deep', 'wake', 'impeach', 'wall', 'potenti', 'proper', 'press', 'adam', 'till', 'doubt', 'gold'}

mng2 - mng1 : {'advic', 'decemb', 'minist', 'teach', 'engin', 'window', 'octob', 'sever', 'sourc', 'draw', 'co', 'dress', 'novemb', 'mobil', 'engag', 'virtual', 'howev', 'rain', 'employe', 'creativ', 'beyond', 'seat', 'afternoon', 'mix', 'loss', 'ha', 'repres', 'thursday', 'june', 'festiv', 'juli', 'regist', 'struggl', 'relationship', 'journey', 'invit', 'strategi'}


For 10000 most frequent words we have

In [187]:
print("length of intersection :", len (frequent_words_mng1 & frequent_words_mng2))
print("length of mng1 - mng2 :",len(frequent_words_mng1 - frequent_words_mng2))
print("length of mng2 - mng1 :",len(frequent_words_mng2 - frequent_words_mng1))

length of intersection : 8927
length of mng1 - mng2 : 1073
length of mng2 - mng1 : 1073


For 100000 most frequent words we have

In [185]:
print("length of intersection :", len (frequent_words_mng1 & frequent_words_mng2))
print("length of mng1 - mng2 :",len(frequent_words_mng1 - frequent_words_mng2))
print("length of mng2 - mng1 :",len(frequent_words_mng2 - frequent_words_mng1))

length of intersection : 59999
length of mng1 - mng2 : 40001
length of mng1 - mng1 : 40001


As it is seen, when number of most frequent words increases, we have more different words

for 100000 most frequent words we have

In [79]:
CHI2_mean=np.mean(chi2)
CHI2_std=np.std(chi2)
P_VALUE_mean=np.mean(p_value)
P_VALUE_std=np.std(p_value)

print('chi2:',chi2)
print('CHI2_mean:',CHI2_mean)
print('CHI2_std:',CHI2_std)
print('p_value:',p_value)
print('P_VALUE_mean:',P_VALUE_mean)
print('P_VALUE_std:',P_VALUE_std)

chi2: [589746.8265019797, 590930.5393329542, 590975.1738484727, 591175.4506700744, 589886.1950987233, 590904.9389806322, 591716.0802972757, 591012.8661516039, 591671.4193496362, 591113.5319510122]
CHI2_mean: 590913.3022182364
CHI2_std: 613.0033241908357
p_value: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
P_VALUE_mean: 0.0
P_VALUE_std: 0.0


In [80]:
print('\n 20 most frequnt words in subcorpus1:')
print(key_list_mng1[0:20])

print('\n 20 most frequnt words in subcorpus2:')
print(key_list_mng2[0:20])


 20 most frequnt words in subcorpus1:
['rt', '’', "'s", 'get', 'like', "n't", 'one', 'go', 'time', 'amp', 'day', 'thank', 'good', 'peopl', 'year', 'see', 'love', 'look', 'know', 'new']

 20 most frequnt words in subcorpus2:
['rt', '’', "'s", 'get', 'like', 'one', "n't", 'go', 'time', 'amp', 'day', 'thank', 'year', 'peopl', 'love', 'good', 'look', 'see', 'make', 'new']


for 10000 most frequent words we have

In [76]:
CHI2_mean=np.mean(chi2)
CHI2_std=np.std(chi2)
P_VALUE_mean=np.mean(p_value)
P_VALUE_std=np.std(p_value)

print('chi2:',chi2)
print('CHI2_mean:',CHI2_mean)
print('CHI2_std:',CHI2_std)
print('p_value:',p_value)
print('P_VALUE_mean:',P_VALUE_mean)
print('P_VALUE_std:',P_VALUE_std)

chi2: [188231.50521356682, 189139.86373553492, 188572.10254375092, 188357.70851842983, 188456.480721872, 189324.18194097356, 188497.31824392098, 188071.64951424135, 188645.69371089223, 188237.23556281443]
CHI2_mean: 188553.3739705997
CHI2_std: 378.60346304089353
p_value: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
P_VALUE_mean: 0.0
P_VALUE_std: 0.0


for 1000 most frequent words we have

In [74]:
CHI2_mean=np.mean(chi2)
CHI2_std=np.std(chi2)
P_VALUE_mean=np.mean(p_value)
P_VALUE_std=np.std(p_value)

print('chi2:',chi2)
print('CHI2_mean:',CHI2_mean)
print('CHI2_std:',CHI2_std)
print('p_value:',p_value)
print('P_VALUE_mean:',P_VALUE_mean)
print('P_VALUE_std:',P_VALUE_std)

chi2: [18580.366580647617, 18820.12459943602, 18665.749469208546, 18283.48818343672, 18570.24513711832, 18668.200382648683, 18933.087325838504, 18593.67454844105, 18909.254256157234, 18952.433103774783]
CHI2_mean: 18697.66235867075
CHI2_std: 198.5786443110858
p_value: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
P_VALUE_mean: 0.0
P_VALUE_std: 0.0


for 500 most frequent words we have

In [72]:
CHI2_mean=np.mean(chi2)
CHI2_std=np.std(chi2)
P_VALUE_mean=np.mean(p_value)
P_VALUE_std=np.std(p_value)

print('chi2:',chi2)
print('CHI2_mean:',CHI2_mean)
print('CHI2_std:',CHI2_std)
print('p_value:',p_value)
print('P_VALUE_mean:',P_VALUE_mean)
print('P_VALUE_std:',P_VALUE_std)

chi2: [11190.190229974132, 11169.521426528012, 11012.31441850915, 11100.940795496463, 11116.596022458994, 10868.532024616634, 10901.63261883241, 10924.225567823047, 11177.477725276927, 10839.836151684602]
CHI2_mean: 11030.126698120037
CHI2_std: 130.23882518783105
p_value: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
P_VALUE_mean: 0.0
P_VALUE_std: 0.0


for 10 most frequent words we have

In [70]:
CHI2_mean=np.mean(chi2)
CHI2_std=np.std(chi2)
P_VALUE_mean=np.mean(p_value)
P_VALUE_std=np.std(p_value)

print('chi2:',chi2)
print('CHI2_mean:',CHI2_mean)
print('CHI2_std:',CHI2_std)
print('p_value:',p_value)
print('P_VALUE_mean:',P_VALUE_mean)
print('P_VALUE_std:',P_VALUE_std)

chi2: [667.9561455148167, 687.1878508975444, 752.0081011612443, 668.8904772629913, 723.4915692332352, 686.1690438085022, 668.7695077536769, 640.5890770589607, 725.341181181385, 686.4052548482657]
CHI2_mean: 690.6808208720623
CHI2_std: 31.73369961775822
p_value: [5.334527812476747e-138, 3.926473017781104e-142, 4.5195569939765685e-156, 3.3599031418070885e-138, 6.148659251850639e-150, 6.501074339166632e-142, 3.5671457530002225e-138, 4.0400360892654343e-132, 2.4604386875751727e-150, 5.783812426053629e-142]
P_VALUE_mean: 4.040048352463278e-133
P_VALUE_std: 1.212010418007844e-132


**For *intra similarity* for mng set based on *screen names* we have** 

In [266]:
names_uniques_mng=list(np.unique(names_mng))
names_mng_arrray = np.array(names_mng)

chi2=[]
p_value=[]

most_frequent=100000

for kk in range(3):

    mng1_names=random.sample(names_uniques_mng,floor(len(names_uniques_mng)/2))
    mng1_ind=[]
    mng2_ind=[]

    for i in mng1_names:
        mng1_ind.extend(list( np.where(names_mng_arrray==i)[0] ))

    mng2_ind=list ( set( range(0,len(mng_cleaned)) ) - set(mng1_ind) )

#    print(len(mng2_ind)+len(mng1_ind))
#    print(len(names_mng))
    
    mng1_tweets=[]
    mng2_tweets=[]

    for i in mng1_ind:
        mng1_tweets.append(mng_cleaned[i])

    for i in mng2_ind:
        mng2_tweets.append(mng_cleaned[i])
    
           
    mng1_words_random=[]
    for tweet in mng1_tweets:
        mng1_words_random += tweet.split()  
    
    mng2_words_random=[]
    for tweet in mng2_tweets:
        mng2_words_random += tweet.split()  
    
    
    all_words=mng1_words_random+mng2_words_random
    
#    rang=range(len(mng_words))
#    slice1_ind=random.sample(rang,floor(len(mng_words)/4))
#    temp=list( set(rang) - set(slice1_ind)  ) # list(a.diference(b))
#    slice2_ind=random.sample(temp,floor(len(temp)/3))
    
#    mng1_words_random=[]
#    for ii in slice1_ind:
#        mng1_words_random.append(mng_words[ii])
        
#    mng2_words_random=[]
#    for ii in slice2_ind:
#        mng2_words_random.append(mng_words[ii])
        
#    all_words=mng_words
    
    
    d_mng1 = dict()
    for c in mng1_words_random:
        if c not in d_mng1:
            d_mng1[c] = 1
        else:
            d_mng1[c] = d_mng1[c] + 1

    d_mng1_new=dict(sorted(d_mng1.items(), key=lambda x: x[1], reverse=True))
    key_list_mng1=list(d_mng1_new.keys())
    values_list_mng1=list(d_mng1_new.values())
    frequent_words_mng1=set(list(d_mng1_new.keys())[0:most_frequent])

#    print(list(d_mng1_new.keys())[0:most_frequent])
#    print(list(d_mng1_new.values())[0:most_frequent]) 
#    print()
    
    d_mng2 = dict()
    for c in mng2_words_random:
        if c not in d_mng2:
            d_mng2[c] = 1
        else:
            d_mng2[c] = d_mng2[c] + 1
            
#    print(list(d_mng2_new.keys())[0:most_frequent])
#    print(list(d_mng2_new.values())[0:most_frequent])   
#    print()

    d_mng2_new=dict(sorted(d_mng2.items(), key=lambda x: x[1], reverse=True))
    key_list_mng2=list(d_mng2_new.keys())
    values_list_mng2=list(d_mng2_new.values())
    frequent_words_mng2=set(list(d_mng2_new.keys())[0:most_frequent])

#    print(list(d_all_new.keys())[0:most_frequent])
#    print(list(d_all_new.values())[0:most_frequent]) 
#    print()

    d_all = dict()
    for c in all_words:
        if c not in d_all:
            d_all[c] = 1
        else:
            d_all[c] = d_all[c] + 1

    d_all_new=dict(sorted(d_all.items(), key=lambda x: x[1], reverse=True))
    key_list_all=list(d_all_new.keys())
    values_list_all=list(d_all_new.values())
        

    frequent=key_list_all[0:most_frequent] # 
#    print(frequent)


    text1_freq_values=[]
    text2_freq_values=[]

    for ii in frequent:

        try: 
            text1_freq_values.append(d_mng1_new[ii])
        except:
            text1_freq_values.append(0)
        try:    
            text2_freq_values.append(d_mng2_new[ii])
        except:
            text2_freq_values.append(0)
        
        
#    print(text1_freq_values)  
#    print(text2_freq_values)  

    chi2_stat, p_val, dof, ex = stats.chi2_contingency([text1_freq_values,text2_freq_values])
    chi2.append(chi2_stat)
    p_value.append(p_val)
    
    
    

for 10 most frequent words we have

In [245]:
print("intersection :", frequent_words_mng1 & frequent_words_mng2)
print()
print("mng1 - mng2 :",frequent_words_mng1 - frequent_words_mng2)
print()
print("mng2 - mng1 :",frequent_words_mng2 - frequent_words_mng1)

intersection : {'go', 'time', 'rt', 'get', 'amp', "'s", "n't", 'like', 'one', '’'}

mng1 - mng2 : set()

mng2 - mng1 : set()


In [246]:
CHI2_mean=np.mean(chi2)
CHI2_std=np.std(chi2)
P_VALUE_mean=np.mean(p_value)
P_VALUE_std=np.std(p_value)

print('chi2:',chi2)
print('CHI2_mean:',CHI2_mean)
print('CHI2_std:',CHI2_std)
print('p_value:',p_value)
print('P_VALUE_mean:',P_VALUE_mean)
print('P_VALUE_std:',P_VALUE_std)

chi2: [619.5981551475755, 978.6924895832485, 337.66668763536666]
CHI2_mean: 645.3191107887302
CHI2_std: 262.32892248733833
p_value: [1.300243272041132e-127, 6.772512269953577e-205, 2.6068304538820288e-67]
P_VALUE_mean: 8.68943484627343e-68
P_VALUE_std: 1.2288716608957254e-67


for 500 most frequent words we have

In [254]:
print("length of intersection :", len(frequent_words_mng1 & frequent_words_mng2))
print()
print("mng1 - mng2 :",frequent_words_mng1 - frequent_words_mng2)
print()
print("mng2 - mng1 :",frequent_words_mng2 - frequent_words_mng1)

length of intersection : 485

mng1 - mng2 : {'forget', 'eye', 'light', 'goe', 'hi', 'inspir', 'featur', 'bless', 'secur', 'im', 'blog', 'saturday', 'room', 'sleep', 'woman'}

mng2 - mng1 : {'major', 'american', 'sport', 'invest', 'act', 'fund', 'fantast', 'london', '—', 'app', 'near', 'price', 'return', 'cup', 'law'}


In [255]:
CHI2_mean=np.mean(chi2)
CHI2_std=np.std(chi2)
P_VALUE_mean=np.mean(p_value)
P_VALUE_std=np.std(p_value)

print('chi2:',chi2)
print('CHI2_mean:',CHI2_mean)
print('CHI2_std:',CHI2_std)
print('p_value:',p_value)
print('P_VALUE_mean:',P_VALUE_mean)
print('P_VALUE_std:',P_VALUE_std)

chi2: [13673.411211268034, 13286.904405736845, 23081.554700226745]
CHI2_mean: 16680.623439077208
CHI2_std: 4528.891522595166
p_value: [0.0, 0.0, 0.0]
P_VALUE_mean: 0.0
P_VALUE_std: 0.0


for 1000 most frequent words we have

In [260]:
print("length of intersection :", len (frequent_words_mng1 & frequent_words_mng2))
print("mng1 - mng2 :",frequent_words_mng1 - frequent_words_mng2)
print("mng2 - mng1 :",frequent_words_mng2 - frequent_words_mng1)

length of intersection : 967
mng1 - mng2 : {'except', 'teach', 'engin', 'felt', 'sever', 'david', 'middl', 'gun', 'ride', 'ice', 'bitch', 'text', 'station', 'financi', 'mix', 'whether', 'boss', 'target', 'repres', 'ha', 'tag', 'suck', 'wake', 'regist', 'hair', 'effort', 'lucki', 'proper', 'oper', 'adam', 'till', 'potenti', 'strategi'}
mng2 - mng1 : {'advic', 'minist', 'https', 'octob', 'click', 'appl', 'fm', 'aw', 'smile', 'nigerian', 'dress', 'progress', 'tree', 'mobil', 'virtual', 'ps', 'feed', 'trend', 'fear', 'yo', 'shout', 'volunt', 'battl', 'search', 'june', 'quot', 'juli', 'struggl', 'wall', 'press', 'lord', 'debat', 'liverpool'}


In [261]:
CHI2_mean=np.mean(chi2)
CHI2_std=np.std(chi2)
P_VALUE_mean=np.mean(p_value)
P_VALUE_std=np.std(p_value)

print('chi2:',chi2)
print('CHI2_mean:',CHI2_mean)
print('CHI2_std:',CHI2_std)
print('p_value:',p_value)
print('P_VALUE_mean:',P_VALUE_mean)
print('P_VALUE_std:',P_VALUE_std)

chi2: [27949.95875515436, 23320.17260019394, 22876.969214769306]
CHI2_mean: 24715.700190039202
CHI2_std: 2294.112560407406
p_value: [0.0, 0.0, 0.0]
P_VALUE_mean: 0.0
P_VALUE_std: 0.0


for 10000 most frequent words we have

In [264]:
print("length of intersection :", len (frequent_words_mng1 & frequent_words_mng2))
print("length of mng1 - mng2 :",len(frequent_words_mng1 - frequent_words_mng2))
print("length of mng2 - mng1 :",len(frequent_words_mng2 - frequent_words_mng1))

length of intersection : 8986
length of mng1 - mng2 : 1014
length of mng2 - mng1 : 1014


In [265]:
CHI2_mean=np.mean(chi2)
CHI2_std=np.std(chi2)
P_VALUE_mean=np.mean(p_value)
P_VALUE_std=np.std(p_value)

print('chi2:',chi2)
print('CHI2_mean:',CHI2_mean)
print('CHI2_std:',CHI2_std)
print('p_value:',p_value)
print('P_VALUE_mean:',P_VALUE_mean)
print('P_VALUE_std:',P_VALUE_std)

chi2: [342901.7868000346, 343853.11857659166, 344769.2932023014]
CHI2_mean: 343841.39952630916
CHI2_std: 762.4513286024703
p_value: [0.0, 0.0, 0.0]
P_VALUE_mean: 0.0
P_VALUE_std: 0.0


for 100000 most frequent words we have

In [267]:
print("length of intersection :", len (frequent_words_mng1 & frequent_words_mng2))
print("length of mng1 - mng2 :",len(frequent_words_mng1 - frequent_words_mng2))
print("length of mng2 - mng1 :",len(frequent_words_mng2 - frequent_words_mng1))

length of intersection : 65974
length of mng1 - mng2 : 34026
length of mng2 - mng1 : 34026


In [268]:
CHI2_mean=np.mean(chi2)
CHI2_std=np.std(chi2)
P_VALUE_mean=np.mean(p_value)
P_VALUE_std=np.std(p_value)

print('chi2:',chi2)
print('CHI2_mean:',CHI2_mean)
print('CHI2_std:',CHI2_std)
print('p_value:',p_value)
print('P_VALUE_mean:',P_VALUE_mean)
print('P_VALUE_std:',P_VALUE_std)

chi2: [1048804.4350262857, 1067550.064007554, 1069925.2849947955]
CHI2_mean: 1062093.2613428782
CHI2_std: 9446.51954739531
p_value: [0.0, 0.0, 0.0]
P_VALUE_mean: 0.0
P_VALUE_std: 0.0


**For *intra similarity* for ent set (based on random selection of tweets) we have** 

In [280]:
chi2=[]
p_value=[]

Ent1_words=ent_words[:floor(len(ent_words)/2)]
Ent2_words=ent_words[floor(len(ent_words)/2):]

most_frequent=100000

for kk in range(10):
  
    ent1_words_random=random.sample(Ent1_words,floor(len(Ent1_words)/2))
    ent2_words_random=random.sample(Ent2_words,floor(len(Ent2_words)/2))   
    all_words=ent1_words_random+ent2_words_random
    
#    rang=range(len(ent_words))
#    slice1_ind=random.sample(rang,floor(len(ent_words)/4))
#    temp=list( set(rang) - set(slice1_ind)  ) # list(a.diference(b))
#    slice2_ind=random.sample(temp,floor(len(temp)/3))
    
#    ent1_words_random=[]
#    for ii in slice1_ind:
#        ent1_words_random.append(ent_words[ii])
        
#    ent2_words_random=[]
#    for ii in slice2_ind:
#        ent2_words_random.append(ent_words[ii])
        
#    all_words=ent_words
    
    
    d_ent1 = dict()
    for c in ent1_words_random:
        if c not in d_ent1:
            d_ent1[c] = 1
        else:
            d_ent1[c] = d_ent1[c] + 1

    d_ent1_new=dict(sorted(d_ent1.items(), key=lambda x: x[1], reverse=True))
    key_list_ent1=list(d_ent1_new.keys())
    values_list_ent1=list(d_ent1_new.values())
    frequent_words_ent1=set(list(d_ent1_new.keys())[0:most_frequent])

#    print(list(d_ent1_new.keys())[0:10])
#    print(list(d_ent1_new.values())[0:10]) 
#    print()
    
    d_ent2 = dict()
    for c in ent2_words_random:
        if c not in d_ent2:
            d_ent2[c] = 1
        else:
            d_ent2[c] = d_ent2[c] + 1
            
#    print(list(d_ent2_new.keys())[0:most_frequent])
#    print(list(d_ent2_new.values())[0:most_frequent])   
#    print()

    d_ent2_new=dict(sorted(d_ent2.items(), key=lambda x: x[1], reverse=True))
    key_list_ent2=list(d_ent2_new.keys())
    values_list_ent2=list(d_ent2_new.values())
    frequent_words_ent2=set(list(d_ent2_new.keys())[0:most_frequent])
    
#    print(list(d_all_new.keys())[0:most_frequent])
#    print(list(d_all_new.values())[0:most_frequent]) 
#    print()

    d_all = dict()
    for c in all_words:
        if c not in d_all:
            d_all[c] = 1
        else:
            d_all[c] = d_all[c] + 1

    d_all_new=dict(sorted(d_all.items(), key=lambda x: x[1], reverse=True))
    key_list_all=list(d_all_new.keys())
    values_list_all=list(d_all_new.values())
        

    frequent=key_list_all[0:most_frequent] # 
#    print(frequent)


    text1_freq_values=[]
    text2_freq_values=[]

    for ii in frequent:

        try: 
            text1_freq_values.append(d_ent1_new[ii])
        except:
            text1_freq_values.append(0)
        try:    
            text2_freq_values.append(d_ent2_new[ii])
        except:
            text2_freq_values.append(0)
        
        
#    print(text1_freq_values)  
#    print(text2_freq_values)  

    chi2_stat, p_val, dof, ex = stats.chi2_contingency([text1_freq_values,text2_freq_values])
    chi2.append(chi2_stat)
    p_value.append(p_val)

for 10 most frequent words we have 

In [270]:
print("intersection :", frequent_words_ent1 & frequent_words_ent2)
print()
print("ent1 - ent2 :",frequent_words_ent1 - frequent_words_ent2)
print()
print("ent2 - ent1 :",frequent_words_ent2 - frequent_words_ent1)

intersection : {'go', 'rt', 'like', 'get', 'amp', "'s", "n't", 'peopl', 'one', '’'}

ent1 - ent2 : set()

ent2 - ent1 : set()


for 500 most frequent words we have 

In [273]:
print("length of intersection :",len( frequent_words_ent1 & frequent_words_ent2))
print()
print("ent1 - ent2 :",frequent_words_ent1 - frequent_words_ent2)
print()
print("ent2 - ent1 :",frequent_words_ent2 - frequent_words_ent1)

length of intersection : 476

ent1 - ent2 : {'age', 'facebook', 'space', 'career', 'uk', 'pictur', 'yeah', 'damn', 'w', 'christma', 'earn', 'ok', 'took', 'bitcoin', 'quot', 'total', 'sleep', 'rest', 'date', 'strong', 'relationship', 'sorri', 'control', 'came'}

ent2 - ent1 : {'song', 'googl', 'african', 'current', 'democrat', 'forc', '\u2066', 'podcast', 'rule', 'entrepreneurship', 'fail', 'blog', 'content', 'short', 'dey', 'intern', 'global', 'award', 'cours', 'announc', 'special', 'perform', 'program', 'photo'}


for 1000 most frequent words we have 

In [277]:
print("length of intersection :", len(frequent_words_ent1 & frequent_words_ent2))
print()
print("ent1 - ent2 :",frequent_words_ent1 - frequent_words_ent2)
print()
print("ent2 - ent1 :",frequent_words_ent2 - frequent_words_ent1)

length of intersection : 971

ent1 - ent2 : {'justic', 'hustl', 'confid', 'origin', 'everyday', 'tho', 'publish', 'eu', 'throw', 'vision', 'daughter', 'ball', 'ppl', 'gun', 'dress', 'incom', 'environ', 'transform', 'bar', 'loss', 'pro', 'broke', 'construct', 'pic', 'door', 'film', 'coupl', 'amazon', 'congress'}

ent2 - ent1 : {'grate', 's…', 'imag', 'lesson', 'fli', 'green', 'oil', 'simpli', 'bbnaija', 'color', 'anim', 'politician', 'contact', 'to…', 'mobil', 'passion', 'session', 'north', 'climat', 'target', 'download', 'legal', 'wine', 'corpor', 'applic', 'ghana', 'doctor', 'gold', 'financ'}


for 10000 most frequent words we have 

In [279]:
print("length of intersection :", len (frequent_words_ent1 & frequent_words_ent2))
print("length of ent1 - ent2 :",len(frequent_words_ent1 - frequent_words_ent2))
print("length of ent2 - ent1 :",len(frequent_words_ent2 - frequent_words_ent1))

length of intersection : 8868
length of ent1 - ent2 : 1132
length of ent2 - ent1 : 1132


for 100000 most frequent words we have 

In [281]:
print("length of intersection :", len (frequent_words_ent1 & frequent_words_ent2))
print("length of ent1 - ent2 :",len(frequent_words_ent1 - frequent_words_ent2))
print("length of ent2 - ent1 :",len(frequent_words_ent2 - frequent_words_ent1))

length of intersection : 59810
length of ent1 - ent2 : 40190
length of ent2 - ent1 : 40190


for 100000 most  frequent words we have

In [65]:
CHI2_mean=np.mean(chi2)
CHI2_std=np.std(chi2)
P_VALUE_mean=np.mean(p_value)
P_VALUE_std=np.std(p_value)

print('chi2:',chi2)
print('CHI2_mean:',CHI2_mean)
print('CHI2_std:',CHI2_std)
print('p_value:',p_value)
print('P_VALUE_mean:',P_VALUE_mean)
print('P_VALUE_std:',P_VALUE_std)

chi2: [576208.4150456071, 574661.8505981333, 577581.1059035398, 575276.0820123482, 575063.2360434899, 576179.0066607032, 575651.6195951918, 576315.3709312071, 575787.5643277803, 576799.802210231]
CHI2_mean: 575952.4053328232
CHI2_std: 815.5115925839615
p_value: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
P_VALUE_mean: 0.0
P_VALUE_std: 0.0


In [68]:
print('\n 20 most frequnt words in subcorpus1:')
print(key_list_ent1[0:20])

print('\n 20 most frequnt words in subcorpus2:')
print(key_list_ent2[0:20])


 20 most frequnt words in subcorpus1:
['rt', '’', "'s", "n't", 'get', 'like', 'one', 'amp', 'peopl', 'go', 'time', 'make', 'know', 'thank', 'need', 'day', 'love', 'year', 'new', 'see']

 20 most frequnt words in subcorpus2:
['rt', '’', "'s", "n't", 'get', 'like', 'amp', 'one', 'peopl', 'go', 'time', 'make', 'know', 'need', 'day', 'thank', 'year', 'see', 'love', 'new']


for 10000 most frequent words we have

In [63]:
CHI2_mean=np.mean(chi2)
CHI2_std=np.std(chi2)
P_VALUE_mean=np.mean(p_value)
P_VALUE_std=np.std(p_value)

print('chi2:',chi2)
print('CHI2_mean:',CHI2_mean)
print('CHI2_std:',CHI2_std)
print('p_value:',p_value)
print('P_VALUE_mean:',P_VALUE_mean)
print('P_VALUE_std:',P_VALUE_std)

chi2: [210116.19711700024, 209477.44093518754, 210069.1024264167, 208229.273002893, 209584.5528074744, 209297.62779014735, 209437.87388814669, 209480.86938200204, 209326.94025798247, 208652.25942542887]
CHI2_mean: 209367.21370326792
CHI2_std: 542.5360397530483
p_value: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
P_VALUE_mean: 0.0
P_VALUE_std: 0.0


for 1000 most frequent words we have

In [61]:
CHI2_mean=np.mean(chi2)
CHI2_std=np.std(chi2)
P_VALUE_mean=np.mean(p_value)
P_VALUE_std=np.std(p_value)

print('chi2:',chi2)
print('CHI2_mean:',CHI2_mean)
print('CHI2_std:',CHI2_std)
print('p_value:',p_value)
print('P_VALUE_mean:',P_VALUE_mean)
print('P_VALUE_std:',P_VALUE_std)

chi2: [18813.08486138571, 18934.35320638869, 18633.896912288754, 19016.082386651433, 18743.274657388363, 18770.132214410987, 18862.696967252097, 18672.39437745939, 18807.136738967485, 18537.27660420405]
CHI2_mean: 18779.032892639698
CHI2_std: 134.7242015306235
p_value: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
P_VALUE_mean: 0.0
P_VALUE_std: 0.0


for 500 most frequent words we have

In [59]:
CHI2_mean=np.mean(chi2)
CHI2_std=np.std(chi2)
P_VALUE_mean=np.mean(p_value)
P_VALUE_std=np.std(p_value)

print('chi2:',chi2)
print('CHI2_mean:',CHI2_mean) 
print('CHI2_std:',CHI2_std)
print('p_value:',p_value)
print('P_VALUE_mean:',P_VALUE_mean)
print('P_VALUE_std:',P_VALUE_std)

chi2: [10325.094093889691, 10455.738005414716, 10442.573969204768, 10438.432011415407, 10231.586420606523, 10742.718978990331, 10514.667883942668, 10321.467940775769, 10145.641933712534, 10177.019247524517]
CHI2_mean: 10379.49404854769
CHI2_std: 169.51581755154095
p_value: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
P_VALUE_mean: 0.0
P_VALUE_std: 0.0


for 10 most frequent words we have

In [57]:
CHI2_mean=np.mean(chi2)
CHI2_std=np.std(chi2)
P_VALUE_mean=np.mean(p_value)
P_VALUE_std=np.std(p_value)

print('chi2:',chi2)
print('CHI2_mean:',CHI2_mean)
print('CHI2_std:',CHI2_std)
print('p_value:',p_value)
print('P_VALUE_mean:',P_VALUE_mean)
print('P_VALUE_std:',P_VALUE_std)

chi2: [606.4668260531298, 506.79421966525723, 546.9847736533058, 541.7317247365197, 598.7047051520464, 602.0134239534013, 518.8164472149542, 547.661744586614, 552.2387270824483, 509.29640086239715]
CHI2_mean: 553.0708992960075
CHI2_std: 35.71500314913203
p_value: [8.57042817861445e-125, 2.0170250683029095e-103, 4.931768441929706e-112, 6.592855502058916e-111, 3.9720128378974735e-123, 7.742490893518853e-124, 5.365562185531523e-106, 3.5308030159353087e-112, 3.68636810789653e-113, 5.872543580594697e-104]
P_VALUE_mean: 2.609645063307674e-104
P_VALUE_std: 6.109224059432716e-104


**For *intra similarity* for ent set based on screen names we have** 

In [15]:
names_uniques_ent=list(np.unique(names_ent))
names_ent_arrray = np.array(names_ent)
 
chi2=[]
p_value=[]

most_frequent=100000

for kk in range(3):
  
    ent1_names=random.sample(names_uniques_ent,floor(len(names_uniques_ent)/2))
    ent1_ind=[]
    ent2_ind=[]

    for i in ent1_names:
        ent1_ind.extend(list( np.where(names_ent_arrray==i)[0] ))

    ent2_ind=list ( set( range(0,len(ent_cleaned)) ) - set(ent1_ind) )

#    print(len(mentng2_ind)+len(ent1_ind))
#    print(len(names_ent))
    
    ent1_tweets=[]
    ent2_tweets=[]

    for i in ent1_ind:
        ent1_tweets.append(ent_cleaned[i])

    for i in ent2_ind:
        ent2_tweets.append(ent_cleaned[i])
        
        
           
    ent1_words_random=[]
    for tweet in ent1_tweets:
        ent1_words_random += tweet.split()  
    
    ent2_words_random=[]
    for tweet in ent2_tweets:
        ent2_words_random += tweet.split() 
        
        
    all_words=ent1_words_random+ent2_words_random
    
#    rang=range(len(ent_words))
#    slice1_ind=random.sample(rang,floor(len(ent_words)/4))
#    temp=list( set(rang) - set(slice1_ind)  ) # list(a.diference(b))
#    slice2_ind=random.sample(temp,floor(len(temp)/3))
    
#    ent1_words_random=[]
#    for ii in slice1_ind:
#        ent1_words_random.append(ent_words[ii])
        
#    ent2_words_random=[]
#    for ii in slice2_ind:
#        ent2_words_random.append(ent_words[ii])
        
#    all_words=ent_words
    
    
    d_ent1 = dict()
    for c in ent1_words_random:
        if c not in d_ent1:
            d_ent1[c] = 1
        else:
            d_ent1[c] = d_ent1[c] + 1

    d_ent1_new=dict(sorted(d_ent1.items(), key=lambda x: x[1], reverse=True))
    key_list_ent1=list(d_ent1_new.keys())
    values_list_ent1=list(d_ent1_new.values())
    frequent_words_ent1=set(list(d_ent1_new.keys())[0:most_frequent])

#    print(list(d_ent1_new.keys())[0:10])
#    print(list(d_ent1_new.values())[0:10]) 
#    print()
    
    d_ent2 = dict()
    for c in ent2_words_random:
        if c not in d_ent2:
            d_ent2[c] = 1
        else:
            d_ent2[c] = d_ent2[c] + 1
            
#    print(list(d_ent2_new.keys())[0:most_frequent])
#    print(list(d_ent2_new.values())[0:most_frequent])   
#    print()

    d_ent2_new=dict(sorted(d_ent2.items(), key=lambda x: x[1], reverse=True))
    key_list_ent2=list(d_ent2_new.keys())
    values_list_ent2=list(d_ent2_new.values())
    frequent_words_ent2=set(list(d_ent2_new.keys())[0:most_frequent])

#    print(list(d_all_new.keys())[0:most_frequent])
#    print(list(d_all_new.values())[0:most_frequent]) 
#    print()

    d_all = dict()
    for c in all_words:
        if c not in d_all:
            d_all[c] = 1
        else:
            d_all[c] = d_all[c] + 1

    d_all_new=dict(sorted(d_all.items(), key=lambda x: x[1], reverse=True))
    key_list_all=list(d_all_new.keys())
    values_list_all=list(d_all_new.values())
        

    frequent=key_list_all[0:most_frequent] # 
#    print(frequent)


    text1_freq_values=[]
    text2_freq_values=[]

    for ii in frequent:

        try: 
            text1_freq_values.append(d_ent1_new[ii])
        except:
            text1_freq_values.append(0)
        try:    
            text2_freq_values.append(d_ent2_new[ii])
        except:
            text2_freq_values.append(0)
        
        
#    print(text1_freq_values)  
#    print(text2_freq_values)  

    chi2_stat, p_val, dof, ex = stats.chi2_contingency([text1_freq_values,text2_freq_values])
    chi2.append(chi2_stat)
    p_value.append(p_val)

for 10 most frequent words we have

In [283]:
print("intersection :", frequent_words_ent1 & frequent_words_ent2)
print("ent1 - ent2 :",frequent_words_ent1 - frequent_words_ent2)
print("ent2 - ent1 :",frequent_words_ent2 - frequent_words_ent1)

intersection : {'go', 'rt', 'like', 'get', 'amp', "'s", "n't", 'peopl', 'one', '’'}
ent1 - ent2 : set()
ent2 - ent1 : set()


In [284]:
CHI2_mean=np.mean(chi2)
CHI2_std=np.std(chi2)
P_VALUE_mean=np.mean(p_value)
P_VALUE_std=np.std(p_value)

print('chi2:',chi2)
print('CHI2_mean:',CHI2_mean)
print('CHI2_std:',CHI2_std)
print('p_value:',p_value)
print('P_VALUE_mean:',P_VALUE_mean)
print('P_VALUE_std:',P_VALUE_std)

chi2: [182.79885432469325, 462.4556873005797, 1246.1615650840884]
CHI2_mean: 630.4720355697872
CHI2_std: 450.07938827328525
p_value: [1.318731166211504e-34, 6.224644932152249e-94, 1.3096577842596867e-262]
P_VALUE_mean: 4.395770554038346e-35
P_VALUE_std: 6.216558334601324e-35


for 500 most frequent words we have

In [297]:
print("length of intersection :", len (frequent_words_ent1 & frequent_words_ent2))
print("ent1 - ent2 :",frequent_words_ent1 - frequent_words_ent2)
print(" ent2 - ent1 :",frequent_words_ent2 - frequent_words_ent1)

length of intersection : 477
ent1 - ent2 : {'facebook', 'space', 'googl', 'african', 'travel', 'yesterday', 'south', 'ceo', 'damn', 'podcast', 'entrepreneurship', 'blog', 'content', 'w', 'short', 'earn', 'dey', 'quot', 'cost', 'strong', 'motiv', 'program', 'photo'}
 ent2 - ent1 : {'major', 'age', 'releas', 'deserv', 'forc', 'pictur', '\u2066', 'articl', 'rule', 'class', 'protect', 'christma', 'creativ', 'global', 'total', 'pray', 'date', 'announc', 'special', 'worth', 'inform', 'dm', 'came'}


In [298]:
CHI2_mean=np.mean(chi2)
CHI2_std=np.std(chi2)
P_VALUE_mean=np.mean(p_value)
P_VALUE_std=np.std(p_value)

print('chi2:',chi2)
print('CHI2_mean:',CHI2_mean)
print('CHI2_std:',CHI2_std)
print('p_value:',p_value)
print('P_VALUE_mean:',P_VALUE_mean)
print('P_VALUE_std:',P_VALUE_std)

chi2: [17386.118028532743, 14952.065389378684, 21635.82656040213]
CHI2_mean: 17991.33665943785
CHI2_std: 2761.989988060078
p_value: [0.0, 0.0, 0.0]
P_VALUE_mean: 0.0
P_VALUE_std: 0.0


for 1000 most frequent words we have

In [302]:
print("length of intersection :", len (frequent_words_ent1 & frequent_words_ent2))
print("ent1 - ent2 :",frequent_words_ent1 - frequent_words_ent2)
print(" ent2 - ent1 :",frequent_words_ent2 - frequent_words_ent1)

length of intersection : 973
ent1 - ent2 : {'confid', 'involv', 'oil', 'tho', 'simpli', 'publish', 'eu', 'vision', 'contact', 'mobil', 'passion', 'north', 'whether', 'seek', 'loss', 'legal', 'corpor', 'statement', 'de', 'brexit', 'broke', 'awar', 'film', 'channel', 'round', 'ft', 'doubt'}
 ent2 - ent1 : {'hustl', 'wit', 's…', 'imag', 'origin', 'fli', 'stress', 'color', 'site', 'politician', 'execut', 'to…', 'forev', 'incom', 'environ', 'l', 'master', 'transform', 'wine', 'debt', 'pro', 'construct', 'digitalmarket', 'tune', 'amazon', 'gold', 'financ'}


In [303]:
CHI2_mean=np.mean(chi2)
CHI2_std=np.std(chi2)
P_VALUE_mean=np.mean(p_value)
P_VALUE_std=np.std(p_value)

print('chi2:',chi2)
print('CHI2_mean:',CHI2_mean)
print('CHI2_std:',CHI2_std)
print('p_value:',p_value)
print('P_VALUE_mean:',P_VALUE_mean)
print('P_VALUE_std:',P_VALUE_std)

chi2: [31500.899954195218, 38536.08550424666, 32863.48058337947]
CHI2_mean: 34300.15534727378
CHI2_std: 3046.4716984101715
p_value: [0.0, 0.0, 0.0]
P_VALUE_mean: 0.0
P_VALUE_std: 0.0


for 10000 most frequent words we have

In [13]:
print("length of intersection :", len (frequent_words_ent1 & frequent_words_ent2))
print("length of ent1 - ent2 :",len(frequent_words_ent1 - frequent_words_ent2))
print("length of ent2 - ent1 :",len(frequent_words_ent2 - frequent_words_ent1))

length of intersection : 8923
length of ent1 - ent2 : 1077
length of ent2 - ent1 : 1077


In [14]:
CHI2_mean=np.mean(chi2)
CHI2_std=np.std(chi2)
P_VALUE_mean=np.mean(p_value)
P_VALUE_std=np.std(p_value)

print('chi2:',chi2)
print('CHI2_mean:',CHI2_mean)
print('CHI2_std:',CHI2_std)
print('p_value:',p_value)
print('P_VALUE_mean:',P_VALUE_mean)
print('P_VALUE_std:',P_VALUE_std) 

chi2: [398049.052280003, 408357.82352811756, 399441.147789167]
CHI2_mean: 401949.3411990958
CHI2_std: 4566.980534057949
p_value: [0.0, 0.0, 0.0]
P_VALUE_mean: 0.0
P_VALUE_std: 0.0


for 100000 most frequent words we have

In [16]:
print("length of intersection :", len (frequent_words_ent1 & frequent_words_ent2))
print("length of ent1 - ent2 :",len(frequent_words_ent1 - frequent_words_ent2))
print("length of ent2 - ent1 :",len(frequent_words_ent2 - frequent_words_ent1))

length of intersection : 66265
length of ent1 - ent2 : 33735
length of ent2 - ent1 : 33735


In [17]:
CHI2_mean=np.mean(chi2)
CHI2_std=np.std(chi2)
P_VALUE_mean=np.mean(p_value)
P_VALUE_std=np.std(p_value)

print('chi2:',chi2)
print('CHI2_mean:',CHI2_mean)
print('CHI2_std:',CHI2_std)
print('p_value:',p_value)
print('P_VALUE_mean:',P_VALUE_mean)
print('P_VALUE_std:',P_VALUE_std)

chi2: [1065781.5214649732, 1049217.985150508, 1042422.4586670494]
CHI2_mean: 1052473.9884275103
CHI2_std: 9810.287855247478
p_value: [0.0, 0.0, 0.0]
P_VALUE_mean: 0.0
P_VALUE_std: 0.0


There are no inter similarity (between ent and mng sets) and intra similarity (for both mng and ent sets based on random selection of tweets and also based on screen names). Even the difference of frequent words of sets becomes more when the number of most frequent words increases.

In [ ]:
a1=[20, 6, 30,44]
a2=[180, 34,50,36]
chi2_stat, p_val, dof, ex = stats.chi2_contingency([a1,a2])
print(chi2_stat)
print(p_val)

In [ ]:
a1=[20, 6,   30, 44]
a2=[21, 6.5, 30, 42]
chi2_stat, p_val, dof, ex = stats.chi2_contingency([a1,a2])
print(chi2_stat)
print(p_val)

In [ ]:
a1=[20, 6,  30, 44]
a2=[40, 12, 60, 88]
chi2_stat, p_val, dof, ex = stats.chi2_contingency([a1,a2])
print(chi2_stat)
print(p_val)